### Read in pickle file

In [28]:
filepath = '../../../downloads/'

In [29]:
import pickle as pickle

In [30]:
with open(filepath + 'final_data.pkl', 'rb') as f:
    df = pickle.load(f)

In [31]:
df.head()

,uri,track,artist,cover image,lyrics
0,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,Mercury: Retrograde,ghostemane,https://i.scdn.co/image/ab67616d0000b27347fa48...,"[Intro]Ask me if I give a fuck about a clique,..."
1,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,Pathology,don kenobi,https://i.scdn.co/image/ab67616d0000b27391b3f8...,[Intro: Don Kenobi]In due time it feels like t...
2,spotify:track:0vSWgAlfpye0WCGeNmuNhy,Symbiote,gizmo,https://i.scdn.co/image/ab67616d0000b2731981f3...,"[Verse]Aye, AyeSmoke a Dutch bluntDon't flick ..."
4,spotify:track:4RYhSQFFlquCTb8XRMJgFU,Venom,$uicideboy$,https://i.scdn.co/image/ab67616d0000b2737811bf...,"[Intro: D.J. Live Wire]Ratatatatatata, boom!Ra..."
5,spotify:track:6fsypiJHyWmeINsOLC1cos,Gatteka,ghostemane,https://i.scdn.co/image/ab67616d0000b273b5ea1c...,"[Intro]Ayy, ayy, uh, uh, uh[Chorus]See, I just..."


In [9]:
data.to_csv('./songs_dataset.csv')

### data processing

In [1]:
import pandas as pd

In [2]:
import re

In [5]:
test = df.loc[0, 'lyrics']

In [6]:
test

"[Intro]Ask me if I give a fuck about a clique, ayyAsk me if I give a fuck about a diss, yuhAsk me if I give a fuck about fame, yuhRecently I just don't give a fuck about a thing, yuh, ayy[Chorus]Ask me if I give a fuck about a diss, ayyAsk me if I give a fuck about a clique, yuhAsk me if I give a fuck about fame, yuhRecently I just don't give a fuck about a thing, yuh[Verse]Ayy, I just decided by the grace of the god PoseidonThat you're so dead to me I dug a hole for you to lie inI'm sick and disowning, all the moments, and the key componentsThat led me to follow hollow promises from empty monadsIt's 11 degrees and I can see my breath so I know I'm breathingBut I got no pulse, I swear to Thelema my heart ain't beatingI better get back to the black hole sunLeaving my gun, I don't need it for this oneI'm finding the silver lining and I'm mining for hopeTryna keep my wrists closed[Bridge]You are toxicMy blood, your lipsYou are toxicMy blood, your lips"

In [22]:
def clean_text(x):
    # remove tags such as [Intro] and [Chorus]
    cleaned_str = re.sub(r'\[.*?\]', '', x)
    # fix typos by adding space before capitalized letter 
    cleaned_str = re.sub(r"(?<=\w)([A-Z])", r" \1", cleaned_str)
    return cleaned_str

In [23]:
df['lyrics_cleaned'] = df['lyrics'].apply(lambda x: clean_text(x))

In [24]:
test = df.loc[0, 'lyrics_cleaned']
print(test)

Ask me if I give a fuck about a clique, ayy Ask me if I give a fuck about a diss, yuh Ask me if I give a fuck about fame, yuh Recently I just don't give a fuck about a thing, yuh, ayy Ask me if I give a fuck about a diss, ayy Ask me if I give a fuck about a clique, yuh Ask me if I give a fuck about fame, yuh Recently I just don't give a fuck about a thing, yuh Ayy, I just decided by the grace of the god Poseidon That you're so dead to me I dug a hole for you to lie in I'm sick and disowning, all the moments, and the key components That led me to follow hollow promises from empty monads It's 11 degrees and I can see my breath so I know I'm breathing But I got no pulse, I swear to Thelema my heart ain't beating I better get back to the black hole sun Leaving my gun, I don't need it for this one I'm finding the silver lining and I'm mining for hope Tryna keep my wrists closed You are toxic My blood, your lips You are toxic My blood, your lips


### Lyrics Summarization Using T5 

In [ ]:
# !pip uninstall transformers
# !pip install transformers

In [11]:
# !pip install sentencepiece

In [7]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

In [8]:
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [9]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [10]:
def summarize_text(x):
    input_str = 'summarize:' + x
    input_ids = tokenizer.encode(input_str, return_tensors='pt', max_length=512, truncation=True)
    summary = t5_model.generate(input_ids, num_beams=4, no_repeat_ngram_size=2, min_length=10, max_length=20)
    summary_str = tokenizer.decode(summary[0], skip_special_tokens=True)
    return summary_str

In [11]:
test = df.loc[0, 'lyrics']

In [12]:
test_summary = summarize_text(test)

In [20]:
df['t5_base_summary'] = df['lyrics_cleaned'].apply(lambda x: summarize_text(x))

In [13]:
df.head()

,uri,track,artist,cover image,lyrics,lyrics_cleaned,t5_base_summary
0,spotify:track:2Vc6NJ9PW9gD9q343XFRKx,Mercury: Retrograde,ghostemane,https://i.scdn.co/image/ab67616d0000b27347fa48...,"[Intro]Ask me if I give a fuck about a clique,...","Ask me if I give a fuck about a clique, ayy As...","ayy, ask me if i give fuck about fame,"
1,spotify:track:7pgJBLVz5VmnL7uGHmRj6p,Pathology,don kenobi,https://i.scdn.co/image/ab67616d0000b27391b3f8...,[Intro: Don Kenobi]In due time it feels like t...,In due time it feels like this all makes sense...,in due time it feels like this all makes sense...
2,spotify:track:0vSWgAlfpye0WCGeNmuNhy,Symbiote,gizmo,https://i.scdn.co/image/ab67616d0000b2731981f3...,"[Verse]Aye, AyeSmoke a Dutch bluntDon't flick ...","Aye, Aye Smoke a Dutch blunt Don't flick the b...",i'm a symbiote I murder and move along I
3,spotify:track:4RYhSQFFlquCTb8XRMJgFU,Venom,$uicideboy$,https://i.scdn.co/image/ab67616d0000b2737811bf...,"[Intro: D.J. Live Wire]Ratatatatatata, boom!Ra...","Ratatatatatata, boom!Ratatatatatata, boom!Two ...",cnn's kelly wallace is a motherfucker who
4,spotify:track:6fsypiJHyWmeINsOLC1cos,Gatteka,ghostemane,https://i.scdn.co/image/ab67616d0000b273b5ea1c...,"[Intro]Ayy, ayy, uh, uh, uh[Chorus]See, I just...","Ayy, ayy, uh, uh, uh See, I just can’t relate ...",mephisto has no control of me Rather be wrappe...


In [14]:
df.to_csv('./songs_dataset_w_summary.csv', index=False)